In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from matplotalt import *

In [2]:
vistext_id_to_mpl_code = np.load("./vistext_id_to_mpl_code.npy", allow_pickle=True).item()
vistext_val_data = pd.read_json("./vistext_data_test.json")
vistext_id_to_captions = defaultdict(list)
vistext_id_to_chart_type = {}
def get_id_to_captions(row):
    vistext_id_to_captions[row["img_id"]].append({"L1": row["caption_L1"], "L2L3": row["caption_L2L3"]})
    vistext_id_to_chart_type[row["img_id"]] = row["L1_properties"][0]

vistext_val_data.apply(get_id_to_captions, axis=1);
np.save("./vistext_test_id_to_captions", vistext_id_to_captions)

In [4]:
#print(len(vistext_id_to_captions))
n_passed = 0
n_errors = 0
#vistext_id_to_captions = np.load("./evaluation/vistext_to_mpl/vistext_id_to_captions.npy", allow_pickle=True).item()
vistext_id_to_matplotalt_captions = {}
for chart_id in tqdm(vistext_id_to_captions.keys()):
    #print(vistext_id_to_mpl_code.keys())
    try:
        plt.clf()
        mpl_code = vistext_id_to_mpl_code[str(chart_id)]
        chart_type = vistext_id_to_chart_type[chart_id]
        exec(mpl_code)
        #print(chart_id)
        #print(mpl_code)
        #plt.show()
        # Only include min and max stats as others (like linear fit, std rarely or do not occur in vistext captions)
        # Also exclude color descriptions since they are different in the matplotlib version of vistext figures
        matplotalt_caption = show_with_alt(desc_level=3, methods=[], stats=["min", "max"], max_color_desc_count=0, return_alt=True)
        vistext_id_to_matplotalt_captions[chart_id] = matplotalt_caption
        n_passed += 1
    except Exception as e:
        #raise e
        print(chart_id, mpl_code)
        n_errors += 1
        #print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")
np.save("./vistext_test_id_to_matplotalt_captions", vistext_id_to_matplotalt_captions)

 86%|████████▌ | 758/882 [00:31<00:04, 29.16it/s]

7360 chart_data = np.array([[23500, 20500], [23500, 15500], [20000, 17000], [16000, 17500], [18000, 17500], [20000, 19000], [21000, 18500], [21000, 19000], [22000, 20000], [22000, 18500], [22500, 20000], [23000, 21000], [22000, 21500], [22000, 20500], [22000, 21000], [21500, 20500], [22500, 20500], [22500, 20500], [22500, 20000], [23000, 20000], [23500, 21000], [23500, 19500], [23000, 20500], [22500, 22000]], dtype=float)
plt.title("Forecasted global oil demand due to the coronavirus pandemic in each month from 2020 to 2021 , by region (in 1,000 barrels per day)")
plt.xlabel("North America")
plt.ylabel("East Asia")
plt.xticks(rotation=45)
plt.bar(chart_data[:, 0], chart_data[:, 1])
plt.xlim(bottom=0.0)
plt.xlim(top=140000.0)

7361 chart_data = np.array([[40500, 20500], [40000, 15500], [36000, 17000], [29000, 17500], [32500, 17500], [33500, 19000], [36000, 18500], [32000, 19000], [37500, 20000], [39000, 18500], [39000, 20000], [40000, 21000], [40500, 21500], [40500, 20500], [41000, 2100

100%|██████████| 882/882 [00:36<00:00, 23.92it/s]

Num passed: 880
Num errors: 2


<Figure size 640x480 with 0 Axes>

In [3]:
vistext_id_to_matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4v_captions = {}

OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
model = "TURBO"
n_errors = 0
n_passed = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    try:
        mpl_code = vistext_id_to_mpl_code[str(chart_id)]
        chart_type = vistext_id_to_chart_type[chart_id]
        exec(mpl_code)
        # gpt4v
        gpt4v_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
                                        desc_level=3, stats=["min", "max"], max_color_desc_count=0,
                                        use_starter_alt_in_prompt=False, use_azure=True,
                                        include_colors=False)
        # gpt4v + starter alt
        #gpt4v_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
        #                                    stats=["min", "max"], max_color_desc_count=0,
        #                                    use_starter_alt_in_prompt=True, use_azure=True,
        #                                    include_colors=False)
        # gpt4v + data table
        #gpt4v_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
        #                                stats=["min", "max"], max_color_desc_count=0,
        #                                use_starter_alt_in_prompt=False, use_azure=True,
        #                                include_colors=False, include_table=True)
        #print(gpt4v_caption)
        plt.clf()
        vistext_id_to_gpt4v_captions[chart_id] = gpt4v_caption
        np.save("./vistext_test_id_to_L3_gpt4v_captions", vistext_id_to_gpt4v_captions)
        n_passed += 1
    except Exception as e:
        #raise e
        print(chart_id)#, mpl_code)
        n_errors += 1
        plt.clf()
        print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")

 56%|█████▌    | 490/880 [54:24<42:01,  6.47s/it]  C:\Users\Kai\Desktop\matplotalt\matplotalt\matplotalt.py:537: UserWarning: Glyph 144 (\x90) missing from current font.
  fig.canvas.draw()
100%|██████████| 880/880 [1:36:34<00:00,  6.58s/it]

Num passed: 880
Num errors: 0


<Figure size 640x480 with 0 Axes>